# RBDS metadata update

Reginaldo K Fukuchi, Feb 2023, reginaldo.fukuchi@ufabc.edu.br

This NB compares the processed files that are in Figshare with the ones that were generated now.

In [1]:
# Prepare environment
import os, glob
import scipy.io as spio
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib notebook

## Import mat file data
### Helper functions to prepare data
https://stackoverflow.com/questions/7008608/scipy-io-loadmat-nested-structures-i-e-dictionaries

In [2]:
def loadmat(filename):
    '''
    this function should be called instead of direct spio.loadmat
    as it cures the problem of not properly recovering python dictionaries
    from mat files. It calls the function check keys to cure all entries
    which are still mat-objects
    '''
    def _check_keys(d):
        '''
        checks if entries in dictionary are mat-objects. If yes
        todict is called to change them to nested dictionaries
        '''
        for key in d:
            if isinstance(d[key], spio.matlab.mat_struct):
                d[key] = _todict(d[key])
        return d

    def _todict(matobj):
        '''
        A recursive function which constructs from matobjects nested dictionaries
        '''
        d = {}
        for strg in matobj._fieldnames:
            elem = matobj.__dict__[strg]
            if isinstance(elem, spio.matlab.mat_struct):
                d[strg] = _todict(elem)
            elif isinstance(elem, np.ndarray):
                d[strg] = _tolist(elem)
            else:
                d[strg] = elem
        return d

    def _tolist(ndarray):
        '''
        A recursive function which constructs lists from cellarrays
        (which are loaded as numpy ndarrays), recursing into the elements
        if they contain matobjects.
        '''
        elem_list = []
        for sub_elem in ndarray:
            if isinstance(sub_elem, spio.matlab.mat_struct):
                elem_list.append(_todict(sub_elem))
            elif isinstance(sub_elem, np.ndarray):
                elem_list.append(_tolist(sub_elem))
            else:
                elem_list.append(sub_elem)
        return elem_list
    data = spio.loadmat(filename, struct_as_record=False, squeeze_me=True)
    return _check_keys(data)

### Flexibility and Strength Measures

In [3]:
dir_data = '../data'

In [4]:
clinicData = loadmat(os.path.join(dir_data, 'clinicData.mat'))

In [5]:
clinicData.keys()

dict_keys(['__header__', '__version__', '__globals__', 'subjID', 'RmuscleM', 'RmuscleSD', 'LmuscleM', 'LmuscleSD', 'MuscleName', 'RflexM', 'RflexSD', 'LflexM', 'LflexSD', 'FlexName', 'Rmuscle', 'Lmuscle', 'Rflex', 'Lflex'])

In [6]:
sub_idxs = clinicData['subjID'].tolist() # original indices of the subjects
stg_cols = clinicData['MuscleName'].tolist() # label of muscle strength measures
flx_cols = clinicData['FlexName'].tolist() # label of muscle flex measures
clinic_cols = ['RThomas', 'LThomas', 'ROber', 'LOber',
               'RHIPABD', 'LHIPABD', 'RHIPEXT', 'LHIPEXT', 'RHIPER', 'LHIPER', 'RHIPIR', 'RHIPER']

In [7]:
c_data = np.empty(shape=(len(sub_idxs), len(clinic_cols))) * np.NaN
c_data[:,[0,2]]=clinicData['Rflex']
c_data[:,[1,3]]=clinicData['Lflex']
c_data[:,[4,6,8,10]]=clinicData['Rmuscle']
c_data[:,[5,7,9,11]]=clinicData['Lmuscle']

In [8]:
df_cdata = pd.DataFrame(data=c_data, columns=clinic_cols)
df_cdata['subID'] = sub_idxs
df_cdata.set_index('subID', inplace=True)

In [9]:
# Get only the rows of new subjects
idx_s41 = np.where(df_cdata.index == 'BMC0041')
df_cdata = df_cdata.iloc[idx_s41[0][0]:,:]
df_cdata

,RThomas,LThomas,ROber,LOber,RHIPABD,LHIPABD,RHIPEXT,LHIPEXT,RHIPER,LHIPER,RHIPIR,RHIPER
subID,,,,,,,,,,,,
BMC0041,3.0,1.0,27.0,26.0,28.666667,24.333333,25.166667,19.433333,11.033333,11.866667,19.033333,19.075000
BMC0042,10.0,9.0,32.0,30.0,28.400000,27.500000,21.950000,23.533333,14.233333,9.666667,20.766667,16.433333
BMC0043,10.0,14.0,34.0,32.0,20.200000,21.125000,17.350000,16.733333,7.900000,7.975000,14.325000,8.800000
BMC0044,10.0,9.0,36.0,35.0,19.966667,18.600000,13.233333,14.500000,11.200000,10.266667,17.466667,10.966667
BMC0045,10.0,12.0,32.0,28.0,25.100000,28.400000,20.425000,16.233333,15.866667,10.966667,17.466667,22.833333
BMC0046,9.0,11.0,37.0,33.0,21.033333,27.133333,20.700000,18.666667,12.575000,8.500000,11.466667,10.400000
BMC0047,6.0,8.0,25.0,28.0,32.066667,35.533333,29.466667,29.350000,14.033333,13.900000,12.366667,12.766667
BMC0048,18.0,17.0,50.0,54.0,29.250000,29.566667,33.566667,31.275000,16.700000,15.566667,23.225000,19.033333
BMC0049,18.0,25.0,30.0,35.0,25.433333,26.133333,25.433333,23.875000,11.533333,7.666667,15.433333,17.033333


### Foot strike index

In [10]:
df_FSI = pd.read_excel(os.path.join(dir_data, 'StrikeIndexResults.xlsx'), 
                       sheet_name='Foot Strike Class', header=1, index_col='ID')

In [11]:
df_FSI = df_FSI.drop('Unnamed: 4', axis=1) # delete column
# Rename column labels
df_FSI.columns = ['RFSI25', 'RFSI35', 'RFSI45', 'LFSI25', 'LFSI35', 'LFSI45']

In [12]:
# Get only the rows of new subjects
idx_s41 = np.where(df_FSI.index == 41)
df_FSI = df_FSI.iloc[idx_s41[0][0]:,:]
df_FSI

,RFSI25,RFSI35,RFSI45,LFSI25,LFSI35,LFSI45
ID,,,,,,
41,Forefoot,Forefoot,Forefoot,Forefoot,Forefoot,Forefoot
42,Midfoot,Midfoot,Rearfoot,Midfoot,Midfoot,Rearfoot
43,NaN,Rearfoot,NaN,NaN,Rearfoot,NaN
44,Forefoot,Forefoot,Forefoot,Forefoot,Forefoot,Forefoot
45,NaN,Midfoot,NaN,NaN,Midfoot,NaN
46,NaN,Rearfoot,NaN,NaN,Rearfoot,NaN
47,NaN,Midfoot,NaN,NaN,Forefoot,NaN
48,NaN,Rearfoot,NaN,NaN,Forefoot,NaN
49,NaN,Midfoot,NaN,NaN,Forefoot,NaN


### Merge clinical data and FSI data

In [13]:
columns = df_FSI.columns.tolist()+df_cdata.columns.tolist()
data = np.hstack((df_FSI.values,df_cdata.values))
df_cFSI = pd.DataFrame(data=data, columns=columns)
df_cFSI

,RFSI25,RFSI35,RFSI45,LFSI25,LFSI35,LFSI45,RThomas,LThomas,ROber,LOber,RHIPABD,LHIPABD,RHIPEXT,LHIPEXT,RHIPER,LHIPER,RHIPIR,RHIPER
0,Forefoot,Forefoot,Forefoot,Forefoot,Forefoot,Forefoot,3.0,1.0,27.0,26.0,28.666667,24.333333,25.166667,19.433333,11.033333,11.866667,19.033333,19.075
1,Midfoot,Midfoot,Rearfoot,Midfoot,Midfoot,Rearfoot,10.0,9.0,32.0,30.0,28.4,27.5,21.95,23.533333,14.233333,9.666667,20.766667,16.433333
2,NaN,Rearfoot,NaN,NaN,Rearfoot,NaN,10.0,14.0,34.0,32.0,20.2,21.125,17.35,16.733333,7.9,7.975,14.325,8.8
3,Forefoot,Forefoot,Forefoot,Forefoot,Forefoot,Forefoot,10.0,9.0,36.0,35.0,19.966667,18.6,13.233333,14.5,11.2,10.266667,17.466667,10.966667
4,NaN,Midfoot,NaN,NaN,Midfoot,NaN,10.0,12.0,32.0,28.0,25.1,28.4,20.425,16.233333,15.866667,10.966667,17.466667,22.833333
5,NaN,Rearfoot,NaN,NaN,Rearfoot,NaN,9.0,11.0,37.0,33.0,21.033333,27.133333,20.7,18.666667,12.575,8.5,11.466667,10.4
6,NaN,Midfoot,NaN,NaN,Forefoot,NaN,6.0,8.0,25.0,28.0,32.066667,35.533333,29.466667,29.35,14.033333,13.9,12.366667,12.766667
7,NaN,Rearfoot,NaN,NaN,Forefoot,NaN,18.0,17.0,50.0,54.0,29.25,29.566667,33.566667,31.275,16.7,15.566667,23.225,19.033333
8,NaN,Midfoot,NaN,NaN,Forefoot,NaN,18.0,25.0,30.0,35.0,25.433333,26.133333,25.433333,23.875,11.533333,7.666667,15.433333,17.033333
9,NaN,Midfoot,NaN,NaN,Forefoot,NaN,15.0,10.0,27.0,26.0,26.3,26.0,23.4,24.7,15.8,11.0,18.325,18.3


### Merge with demographics from google forms

In [14]:
fname_q= os.path.join(r'../data', 'RBDS_google_forms.csv') 
# Import data
info2 = pd.read_csv(fname_q, sep = ',', index_col = 'ID')
# Update df with s041 onwards
info2 = info2.iloc[int(np.where(info2.index=='BMC0041')[0][0]):,:]

In [15]:
col_labels = ['Data de nascimento','Altura (cm)','Massa (kg)','Gênero','Membro inferior dominante',
              'Qual é o seu desempenho na corrida?','Quanto tempo (em meses) você tem praticado corrida de rua regularmente?',
             'Quantas vezes por semana você pratica corrida?','Qual é o tipo de solo (terreno) que você realiza os seus treinos? [Esteira]',
             'Qual é o tipo de solo (terreno) que você realiza os seus treinos? [Asfalto]',
              'Qual é o tipo de solo (terreno) que você realiza os seus treinos? [Grama]',
             'Qual é o tipo de solo (terreno) que você realiza os seus treinos? [Terra]',
             'Qual é o tipo de solo (terreno) que você realiza os seus treinos? [Areia]',
             'Qual é o tipo de solo (terreno) que você realiza os seus treinos? [Concreto]',
             'Qual é o tipo de solo (terreno) que você realiza os seus treinos? [Outro]',
              'Participante de grupo (assessoria) de corrida?','Indique a sua quilometragem semanal?',
              'Qual foi o seu pace médio (min/km) nas últimas provas?',
              'Qual (is) é (são) o (s) tipo (s) de prova que você realizada ultimamente?',
              'Você apresentou lesão (ou dor) em decorrência da prática de corrida que tenha te afastado pelo menos de um treino/prova?',
              'Localização anatômica da lesão', 'Houve diagnóstico nosológico?', 'Caso tenha respondido sim na questão anterior',
              'Data aproximada do inicio dos sintomas', 'Número do tênis de corrida', 'Marca do calçado', 'Modelo do calçado',
              'Número de pares', 'Com que frequência você substitui os ses tênis?', 'Indique o nível de conforto do seu tênis de corrida atual',
              'Você utiliza algum objeto dentro do tênis?']

In [16]:
info3 = info2[col_labels]
# Using drop() function to delete last row
info3 = info3.iloc[:-1,:]
info3.head()

,Data de nascimento,Altura (cm),Massa (kg),Gênero,Membro inferior dominante,Qual é o seu desempenho na corrida?,Quanto tempo (em meses) você tem praticado corrida de rua regularmente?,Quantas vezes por semana você pratica corrida?,Qual é o tipo de solo (terreno) que você realiza os seus treinos? [Esteira],Qual é o tipo de solo (terreno) que você realiza os seus treinos? [Asfalto],...,Houve diagnóstico nosológico?,Caso tenha respondido sim na questão anterior,Data aproximada do inicio dos sintomas,Número do tênis de corrida,Marca do calçado,Modelo do calçado,Número de pares,Com que frequência você substitui os ses tênis?,Indique o nível de conforto do seu tênis de corrida atual,Você utiliza algum objeto dentro do tênis?
ID,,,,,,,,,,,,,,,,,,,,,
BMC0041,5/17/1983,168.5,63.10,Masculino,Direito,Competitivo,84,5x/semana,2,3,...,NaN,NaN,NaN,9,Nike,Streaker Air zoom 6,1,Entre 7 meses e 1 ano,8,Nenhum
BMC0042,4/5/1984,181.2,75.35,Masculino,Direito,Competitivo,24,4x/semana,2,2,...,Sim,"Tendinite t calcâneo D, fasceíte plantar E",9/1/2016,US 9.5,Asics,Nimbus 17,2,Entre 7 meses e 1 ano,8,Nenhum
BMC0043,5/23/1997,176.5,64.70,Masculino,Direito,Apenas recreacional,2,2x/semana,0,2,...,NaN,NaN,NaN,9.5,Nike,Air Relentless4,1,Entre 7 meses e 1 ano,10,Nenhum
BMC0044,6/7/1976,177.2,78.30,Masculino,Direito,Competitivo,264,3x/semana,0,3,...,NaN,NaN,NaN,10.5,Nike,Pegasus,3,Menos de 6 meses,8,Nenhum
BMC0045,3/13/1986,176.7,101.30,Masculino,Direito,Recreacional competitivo,18,4x/semana,0,4,...,Sim,Fasceíte plantar D,11/1/2016,US 10,Asics,Contend 3A,1,Menos de 6 meses,7,Palmilhas


### Calc age based on date and timestamp

In [17]:
from datetime import datetime, date

In [18]:
age = [0] * info3.shape[0]
for d in range(info3.shape[0]):
    born = info3['Data de nascimento'][d]
    today= info2['Timestamp'][d][:-9]
    #Identify given date as date month and year
    born  = datetime.strptime(born, "%m/%d/%Y").date()
    today = datetime.strptime(today, "%m/%d/%Y").date()

    age[d] = today.year - born.year - ((today.month, today.day) < (born.month, born.day))

In [19]:
# Add age columns pandas df
info3.insert(0, 'Age', age)
info3 = info3.drop('Data de nascimento', axis=1) # delete column

### Import RBDSinfo df

In [128]:
info = pd.read_csv(os.path.join(r'../data','RBDSinfo.txt'), 
                   delimiter='\t', index_col='Subject')
info.head()

,FileName,Age,Height,Mass,Gender,Dominance,Level,Experience,SessionsPerWk,Treadmill,...,LThomas,ROber,LOber,RHIPABD,LHIPABD,RHIPEXT,LHIPEXT,RHIPER,LHIPER,RHIPIR
Subject,,,,,,,,,,,,,,,,,,,,,
1,RBDS001static.txt,22,181.0,62.0,M,R,Competitive,4,3,0,...,11,43,40,16.8,21.575,16.575,21.675,9.666667,7.25,10.966667
1,RBDS001runT25markers.txt,22,181.0,62.0,M,R,Competitive,4,3,0,...,11,43,40,16.8,21.575,16.575,21.675,9.666667,7.25,10.966667
1,RBDS001runT25forces.txt,22,181.0,62.0,M,R,Competitive,4,3,0,...,11,43,40,16.8,21.575,16.575,21.675,9.666667,7.25,10.966667
1,RBDS001runT35markers.txt,22,181.0,62.0,M,R,Competitive,4,3,0,...,11,43,40,16.8,21.575,16.575,21.675,9.666667,7.25,10.966667
1,RBDS001runT35forces.txt,22,181.0,62.0,M,R,Competitive,4,3,0,...,11,43,40,16.8,21.575,16.575,21.675,9.666667,7.25,10.966667


#### Replace column names

In [21]:
print(info.columns)

Index(['FileName', 'Age', 'Height', 'Mass', 'Gender', 'Dominance', 'Level',
       'Experience', 'SessionsPerWk', 'Treadmill', 'Aslphalt', 'Grass',
       'Trail', 'Sand', 'Concrete', 'SurfaceAlt', 'RunGrp', 'Volume', 'Pace',
       'RaceDist', 'Injury', 'InjuryLoc', 'DiagnosticMed', 'Diagnostic',
       'InjuryOnDate', 'ShoeSize', 'ShoeBrand', 'ShoeModel', 'ShoePairs',
       'ShoeChange', 'ShoeComfort', 'ShoeInsert', 'RFSI25', 'RFSI35', 'RFSI45',
       'LFSI25', 'LFSI35', 'LFSI45', 'RThomas', 'LThomas', 'ROber', 'LOber',
       'RHIPABD', 'LHIPABD', 'RHIPEXT', 'LHIPEXT', 'RHIPER', 'LHIPER',
       'RHIPIR'],
      dtype='object')


In [22]:
print(info3.columns.tolist())

['Age', 'Altura (cm)', 'Massa (kg)', 'Gênero', 'Membro inferior dominante', 'Qual é o seu desempenho na corrida?', 'Quanto tempo (em meses) você tem praticado corrida de rua regularmente?', 'Quantas vezes por semana você pratica corrida?', 'Qual é o tipo de solo (terreno) que você realiza os seus treinos? [Esteira]', 'Qual é o tipo de solo (terreno) que você realiza os seus treinos? [Asfalto]', 'Qual é o tipo de solo (terreno) que você realiza os seus treinos? [Grama]', 'Qual é o tipo de solo (terreno) que você realiza os seus treinos? [Terra]', 'Qual é o tipo de solo (terreno) que você realiza os seus treinos? [Areia]', 'Qual é o tipo de solo (terreno) que você realiza os seus treinos? [Concreto]', 'Qual é o tipo de solo (terreno) que você realiza os seus treinos? [Outro]', 'Participante de grupo (assessoria) de corrida?', 'Indique a sua quilometragem semanal?', 'Qual foi o seu pace médio (min/km) nas últimas provas?', 'Qual (is) é (são) o (s) tipo (s) de prova que você realizada ulti

In [23]:
columns = ['Age', 'Height', 'Mass', 'Gender', 'Dominance', 'Level',
       'Experience', 'SessionsPerWk', 'Treadmill', 'Aslphalt', 'Grass',
       'Trail', 'Sand', 'Concrete', 'SurfaceAlt', 'RunGrp', 'Volume', 'Pace',
       'RaceDist', 'Injury', 'InjuryLoc', 'DiagnosticMed', 'Diagnostic',
       'InjuryOnDate', 'ShoeSize', 'ShoeBrand', 'ShoeModel', 'ShoePairs',
       'ShoeChange', 'ShoeComfort', 'ShoeInsert']

In [24]:
info3.columns=columns
info3.head()

,Age,Height,Mass,Gender,Dominance,Level,Experience,SessionsPerWk,Treadmill,Aslphalt,...,DiagnosticMed,Diagnostic,InjuryOnDate,ShoeSize,ShoeBrand,ShoeModel,ShoePairs,ShoeChange,ShoeComfort,ShoeInsert
ID,,,,,,,,,,,,,,,,,,,,,
BMC0041,33,168.5,63.10,Masculino,Direito,Competitivo,84,5x/semana,2,3,...,NaN,NaN,NaN,9,Nike,Streaker Air zoom 6,1,Entre 7 meses e 1 ano,8,Nenhum
BMC0042,32,181.2,75.35,Masculino,Direito,Competitivo,24,4x/semana,2,2,...,Sim,"Tendinite t calcâneo D, fasceíte plantar E",9/1/2016,US 9.5,Asics,Nimbus 17,2,Entre 7 meses e 1 ano,8,Nenhum
BMC0043,19,176.5,64.70,Masculino,Direito,Apenas recreacional,2,2x/semana,0,2,...,NaN,NaN,NaN,9.5,Nike,Air Relentless4,1,Entre 7 meses e 1 ano,10,Nenhum
BMC0044,41,177.2,78.30,Masculino,Direito,Competitivo,264,3x/semana,0,3,...,NaN,NaN,NaN,10.5,Nike,Pegasus,3,Menos de 6 meses,8,Nenhum
BMC0045,31,176.7,101.30,Masculino,Direito,Recreacional competitivo,18,4x/semana,0,4,...,Sim,Fasceíte plantar D,11/1/2016,US 10,Asics,Contend 3A,1,Menos de 6 meses,7,Palmilhas


In [25]:
# Export to csv to modify manually
#info3.to_csv(os.path.join(r'../data','info.csv'))

## Spreadsheet manually translated into English

In [25]:
# Import data already translated
info4 = pd.read_excel(os.path.join(r'../data','Book1.xlsx'))
info4

,ID,Age,Height,Mass,Gender,Dominance,Level,Experience,SessionsPerWk,Treadmill,...,DiagnosticMed,Diagnostic,InjuryOnDate,ShoeSize,ShoeBrand,ShoeModel,ShoePairs,ShoeChange,ShoeComfort,ShoeInsert
0,BMC0041,33,168.5,63.10,M,R,Competitive,84,5,2,...,NaN,NaN,NaT,9.0,Nike,Streaker Air zoom 6,1,Between 7 months and one year,8,None
1,BMC0042,32,181.2,75.35,M,R,Competitive,24,4,2,...,Yes,Aquiles tendinitis,2016-09-01,9.5,Asics,Nimbus 17,2,Between 7 months and one year,8,None
2,BMC0043,19,176.5,64.70,M,R,Recreational,2,2,0,...,NaN,NaN,NaT,9.5,Nike,Air Relentless4,1,Between 7 months and one year,10,None
3,BMC0044,41,177.2,78.30,M,R,Competitive,264,3,0,...,NaN,NaN,NaT,10.5,Nike,Pegasus,3,Less than 6 months,8,None
4,BMC0045,31,176.7,101.30,M,R,Competitive,18,4,0,...,Yes,Plantar fasciitis,2016-11-01,10.0,Asics,Contend 3A,1,Less than 6 months,7,Palmilhas
5,BMC0046,29,179.3,73.50,M,R,Competitive,24,3,2,...,Yes,Shin splints,2017-06-25,9.5,Nike,Free RN distance 2,1,Less than 6 months,8,None
6,BMC0047,46,165.0,65.60,M,R,Competitive,24,4,0,...,NaN,NaN,NaT,8.5,Joma,Fast,3,Less than 6 months,8,None
7,BMC0048,30,192.4,81.00,M,R,Competitive,24,4,1,...,NaN,NaN,NaT,12.0,Mizuno,Wave Enigma 5,2,Less than 6 months,10,None
8,BMC0049,40,164.0,64.50,M,R,Competitive,12,2,0,...,Yes,Chondromalacia patellae,2017-01-01,8.5,Nike,Zoom Winflo 3,1,Between 7 months and one year,8,None
9,BMC0050,40,171.6,65.25,M,R,Competitive,36,6,2,...,NaN,NaN,NaT,9.0,Asics,Kayano,6,Less than 6 months,10,None


## Concat df with demographics and FSI/clinic data

In [26]:
df_o2 = pd.concat([info4, df_cFSI], axis=1)

In [28]:
df_o2.head()

,ID,Age,Height,Mass,Gender,Dominance,Level,Experience,SessionsPerWk,Treadmill,...,ROber,LOber,RHIPABD,LHIPABD,RHIPEXT,LHIPEXT,RHIPER,LHIPER,RHIPIR,RHIPER
0,BMC0041,33,168.5,63.10,M,R,Competitive,84,5,2,...,27.0,26.0,28.666667,24.333333,25.166667,19.433333,11.033333,11.866667,19.033333,19.075
1,BMC0042,32,181.2,75.35,M,R,Competitive,24,4,2,...,32.0,30.0,28.4,27.5,21.95,23.533333,14.233333,9.666667,20.766667,16.433333
2,BMC0043,19,176.5,64.70,M,R,Recreational,2,2,0,...,34.0,32.0,20.2,21.125,17.35,16.733333,7.9,7.975,14.325,8.8
3,BMC0044,41,177.2,78.30,M,R,Competitive,264,3,0,...,36.0,35.0,19.966667,18.6,13.233333,14.5,11.2,10.266667,17.466667,10.966667
4,BMC0045,31,176.7,101.30,M,R,Competitive,18,4,0,...,32.0,28.0,25.1,28.4,20.425,16.233333,15.866667,10.966667,17.466667,22.833333


## Rename and transfer files; and create filenames column

In [29]:
# importing shutil module
import shutil
from tqdm.notebook import tqdm

In [30]:
# Figshare files local directory
figshare_dir_new = r'C:\Users\Reginaldo\Documents\data\CNPq\RBDS_v2\Figshare_update'
fnames = glob.glob(os.path.join(figshare_dir_new,'RBDS0*processed.txt'))

In [31]:
# Get the subject indices
sub_idxs = [fname[-15:-13] for fname in fnames]
sub_idxs

['41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51']

## Get filenames

In [121]:
cols_fn = []
for s,sidx in enumerate(sub_idxs):
    cols_fn.append('RBDS0'+str(29+s)+'static.txt')
    # Get running filenames
    fn_run = glob.glob(os.path.join(figshare_dir_new,'upload','RBDS0'+str(29+s)+'runT*.c3d'))
    for c, cond in enumerate(fn_run):
        fn_mkr = 'RBDS0'+str(29+s)+'runT'+cond[cond.find('T')+1:cond.find('T')+3]+'markers.txt'
        fn_grf = 'RBDS0'+str(29+s)+'runT'+cond[cond.find('T')+1:cond.find('T')+3]+'forces.txt'
        cols_fn.append(fn_mkr)
        cols_fn.append(fn_grf)
    cols_fn.append('RBDS0'+str(29+s)+'static.c3d')
    for c, cond in enumerate(fn_run):
        fn_c3d = 'RBDS0'+str(29+s)+'runT'+cond[cond.find('T')+1:cond.find('T')+3]+'.c3d'
        cols_fn.append(fn_c3d) 
    cols_fn.append('RBDS0'+str(29+s)+'processed.txt')

## Repeat rows of df based on the number of files

In [139]:
data = np.empty(shape=(0,df_o2.shape[1]))
id_subs = []
for sb, sidx in enumerate(sub_idxs):
    matching = [s for s in cols_fn if 'RBDS0'+str(29+sb) in s]
    data=np.append(data, np.repeat(df_o2.values[sb,:], len(matching), axis=0).reshape((len(matching),df_o2.shape[1]), 
                                                                                      order='F'), axis=0)
    id_subs = id_subs+[29+sb]*len(matching)

In [140]:
info5 = pd.DataFrame(data=data,columns=df_o2.columns)

In [141]:
# Third position would be at index 2, because of zero-indexing.
info5.insert(0, 'FileName', cols_fn)
info5.insert(0, 'Subject', id_subs)
info5.drop('ID', axis=1, inplace=True) # Drop ID column
info5.set_index('Subject', inplace=True)

In [142]:
info5 = info5.iloc[:, :-1] # Removing the last column

In [143]:
info5.reset_index(inplace=True)
info5.head()

,Subject,FileName,Age,Height,Mass,Gender,Dominance,Level,Experience,SessionsPerWk,...,LThomas,ROber,LOber,RHIPABD,LHIPABD,RHIPEXT,LHIPEXT,RHIPER,LHIPER,RHIPIR
0,29,RBDS029static.txt,33,168.5,63.1,M,R,Competitive,84,5,...,1.0,27.0,26.0,28.666667,24.333333,25.166667,19.433333,11.033333,11.866667,19.033333
1,29,RBDS029runT25markers.txt,33,168.5,63.1,M,R,Competitive,84,5,...,1.0,27.0,26.0,28.666667,24.333333,25.166667,19.433333,11.033333,11.866667,19.033333
2,29,RBDS029runT25forces.txt,33,168.5,63.1,M,R,Competitive,84,5,...,1.0,27.0,26.0,28.666667,24.333333,25.166667,19.433333,11.033333,11.866667,19.033333
3,29,RBDS029runT35markers.txt,33,168.5,63.1,M,R,Competitive,84,5,...,1.0,27.0,26.0,28.666667,24.333333,25.166667,19.433333,11.033333,11.866667,19.033333
4,29,RBDS029runT35forces.txt,33,168.5,63.1,M,R,Competitive,84,5,...,1.0,27.0,26.0,28.666667,24.333333,25.166667,19.433333,11.033333,11.866667,19.033333


In [129]:
info.reset_index(inplace=True)
info.head()

,Subject,FileName,Age,Height,Mass,Gender,Dominance,Level,Experience,SessionsPerWk,...,LThomas,ROber,LOber,RHIPABD,LHIPABD,RHIPEXT,LHIPEXT,RHIPER,LHIPER,RHIPIR
0,1,RBDS001static.txt,22,181.0,62.0,M,R,Competitive,4,3,...,11,43,40,16.8,21.575,16.575,21.675,9.666667,7.25,10.966667
1,1,RBDS001runT25markers.txt,22,181.0,62.0,M,R,Competitive,4,3,...,11,43,40,16.8,21.575,16.575,21.675,9.666667,7.25,10.966667
2,1,RBDS001runT25forces.txt,22,181.0,62.0,M,R,Competitive,4,3,...,11,43,40,16.8,21.575,16.575,21.675,9.666667,7.25,10.966667
3,1,RBDS001runT35markers.txt,22,181.0,62.0,M,R,Competitive,4,3,...,11,43,40,16.8,21.575,16.575,21.675,9.666667,7.25,10.966667
4,1,RBDS001runT35forces.txt,22,181.0,62.0,M,R,Competitive,4,3,...,11,43,40,16.8,21.575,16.575,21.675,9.666667,7.25,10.966667


In [137]:
info5.shape

(84, 50)

### Concatenate RBDSinfo metadata with newer subjects

In [145]:
RBDSmeta2 = pd.concat([info, info5], axis=0)
RBDSmeta2.set_index('Subject', inplace=True)
RBDSmeta2.head()

,FileName,Age,Height,Mass,Gender,Dominance,Level,Experience,SessionsPerWk,Treadmill,...,LThomas,ROber,LOber,RHIPABD,LHIPABD,RHIPEXT,LHIPEXT,RHIPER,LHIPER,RHIPIR
Subject,,,,,,,,,,,,,,,,,,,,,
1,RBDS001static.txt,22,181.0,62.0,M,R,Competitive,4,3,0,...,11,43,40,16.8,21.575,16.575,21.675,9.666667,7.25,10.966667
1,RBDS001runT25markers.txt,22,181.0,62.0,M,R,Competitive,4,3,0,...,11,43,40,16.8,21.575,16.575,21.675,9.666667,7.25,10.966667
1,RBDS001runT25forces.txt,22,181.0,62.0,M,R,Competitive,4,3,0,...,11,43,40,16.8,21.575,16.575,21.675,9.666667,7.25,10.966667
1,RBDS001runT35markers.txt,22,181.0,62.0,M,R,Competitive,4,3,0,...,11,43,40,16.8,21.575,16.575,21.675,9.666667,7.25,10.966667
1,RBDS001runT35forces.txt,22,181.0,62.0,M,R,Competitive,4,3,0,...,11,43,40,16.8,21.575,16.575,21.675,9.666667,7.25,10.966667


## Export RBDSinfo metadata

In [147]:
#RBDSmeta2.to_csv(os.path.join(figshare_dir_new,'upload','RBDSinfo.txt'), sep='\t')
#RBDSmeta2.to_excel(os.path.join(figshare_dir_new,'upload','RBDSinfo.xlsx'))

## Transfer TXT files

In [61]:
for s,sidx in tqdm(enumerate(sub_idxs), leave=True):
    # PROCESSED TXT FILES
    fn_process_o = os.path.join(figshare_dir_new,'RBDS0'+sidx+'processed.txt')
    fn_process_d = os.path.join(figshare_dir_new,'RBDS0'+str(29+s)+'processed.txt')
    # Copy and paste processed file
    #shutil.copy(fn_process_o, fn_process_d)
    # STATIC TXT FILES
    fn_static_o = os.path.join(figshare_dir_new,'RBDS0'+sidx+'static.txt')
    fn_static_d = os.path.join(figshare_dir_new,'RBDS0'+str(29+s)+'static.txt')
    # Copy and paste processed file
    #shutil.copy(fn_static_o, fn_static_d)
    # RUNNING TXT FILES
    fn_run = glob.glob(os.path.join(figshare_dir_new,'RBDS0'+sidx+'runT*.txt'))
    for fn, fn_r in tqdm(enumerate(fn_run), leave=False):
        fn_run_d = fn_r[:fn_r.find(sidx)]+str(29+s)+fn_r[fn_r.find(sidx)+2:]
        # Copy and paste processed file
        #shutil.copy(fn_r, fn_run_d)

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

...

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [62]:
rba_dir = r'C:\Users\Reginaldo\Documents\data\CNPq\RBDS_v2\RBA'

In [ ]:
fn_run_o = glob.glob(os.path.join(rba_dir,'SUB00'+sidx,'runT*.c3d'))
fn_run_o[0][-11:-5]

### Transfer c3d files

In [63]:
for s,sidx in enumerate(sub_idxs):
    fn_static_o = glob.glob(os.path.join(rba_dir,'SUB00'+sidx,'static.c3d'))
    fn_static_d = os.path.join(figshare_dir_new,'RBDS0'+str(29+s)+'static.c3d')
    # Copy and paste static c3d file
    #shutil.copy(fn_static_o[0], fn_static_d)
    # Running files
    fn_run_o = glob.glob(os.path.join(rba_dir,'SUB00'+sidx,'runT*.c3d'))
    for v in range(len(fn_run_o)):
        fn_run_d = os.path.join(figshare_dir_new,'RBDS0'+str(29+s)+fn_run_o[v][-11:-5]+'.c3d')
        # Copy and paste static c3d file
        #shutil.copy(fn_run_o[v], fn_run_d)
        print(fn_run_d)

C:\Users\Reginaldo\Documents\data\CNPq\RBDS_v2\Figshare_update\RBDS029runT25.c3d
C:\Users\Reginaldo\Documents\data\CNPq\RBDS_v2\Figshare_update\RBDS029runT35.c3d
C:\Users\Reginaldo\Documents\data\CNPq\RBDS_v2\Figshare_update\RBDS029runT45.c3d
C:\Users\Reginaldo\Documents\data\CNPq\RBDS_v2\Figshare_update\RBDS030runT25.c3d
C:\Users\Reginaldo\Documents\data\CNPq\RBDS_v2\Figshare_update\RBDS030runT35.c3d
C:\Users\Reginaldo\Documents\data\CNPq\RBDS_v2\Figshare_update\RBDS030runT45.c3d
C:\Users\Reginaldo\Documents\data\CNPq\RBDS_v2\Figshare_update\RBDS031runT35.c3d
C:\Users\Reginaldo\Documents\data\CNPq\RBDS_v2\Figshare_update\RBDS032runT25.c3d
C:\Users\Reginaldo\Documents\data\CNPq\RBDS_v2\Figshare_update\RBDS032runT35.c3d
C:\Users\Reginaldo\Documents\data\CNPq\RBDS_v2\Figshare_update\RBDS032runT45.c3d
C:\Users\Reginaldo\Documents\data\CNPq\RBDS_v2\Figshare_update\RBDS033runT35.c3d
C:\Users\Reginaldo\Documents\data\CNPq\RBDS_v2\Figshare_update\RBDS034runT35.c3d
C:\Users\Reginaldo\Documents

## Search strings inside files

In [2]:
from tqdm.notebook import tqdm

In [10]:
user_input = r'C:\Users\Reginaldo\Documents\Matlab\scripts'
directory = os.listdir(user_input)

searchstring = 'clinicData.mat'

for fname in tqdm(directory):
    if os.path.isfile(user_input + os.sep + fname):
        # Full path
        f = open(user_input + os.sep + fname, 'r')

        if searchstring in f.read():
            print('found string in file %s' % fname)
        f.close()

  0%|          | 0/162 [00:00<?, ?it/s]

found string in file clinicalMeasuresCalc.m
